In [93]:
import time
import datetime
import os
import yaml
import math
import sys
import gurobipy
import pandas as pd
import numpy as np
from gurobipy import Model, GRB, quicksum

print(f"python version: {sys.version}")
print(f"gurobipy version: {gurobipy.gurobi.version()}")


python version: 3.9.6 (default, Sep 26 2022, 11:37:49) 
[Clang 14.0.0 (clang-1400.0.29.202)]
gurobipy version: (11, 0, 0)


In [94]:
# 添加相對路徑到 sys.path
sys.path.append('../')  
from path_config import INSTANCES_DIR
def load_specific_yaml(filename):
    """
    加載指定的 YAML 檔案。
    
    Parameters:
    filename (str): 在 instances 資料夾中的 YAML 檔案名。
    
    Returns:
    dict: YAML 檔案內容。
    """
    file_path = os.path.join(INSTANCES_DIR, filename)
    with open(file_path, 'r') as file:
        data = yaml.safe_load(file)
    return data

# Load experiments yaml
config = load_specific_yaml('instance_F_4.yaml')
config['lambda_for_G'] = 3e-6
config['M'] = 1000000
config

{'A_opponent_bar': [80, 39, 62, 139, 60, 131],
 'B': [[2, 2, 2],
  [1, 1, 2],
  [4, 4, 1],
  [2, 1, 1],
  [1, 3, 4],
  [3, 2, 2],
  [4, 1, 2],
  [4, 3, 3],
  [3, 3, 1],
  [2, 1, 3],
  [3, 1, 2],
  [3, 1, 4]],
 'C': [1, 2, 1, 3, 3, 2, 3, 1, 1, 1, 3, 2],
 'D': [[38.48,
   29.53,
   41.18,
   42.94,
   23.77,
   25.61,
   8.94,
   7.0,
   53.08,
   63.78,
   54.01,
   31.0],
  [25.71,
   38.08,
   66.53,
   67.9,
   47.42,
   53.54,
   37.12,
   37.11,
   67.23,
   91.05,
   83.38,
   60.67],
  [102.3,
   79.93,
   51.0,
   49.24,
   69.34,
   67.12,
   83.24,
   89.05,
   54.71,
   33.06,
   82.61,
   82.04],
  [16.0,
   11.18,
   43.83,
   44.78,
   25.96,
   35.23,
   24.35,
   30.53,
   40.31,
   68.88,
   74.95,
   52.5],
  [52.81,
   39.62,
   37.34,
   39.32,
   25.63,
   21.02,
   14.21,
   8.06,
   56.92,
   55.95,
   39.0,
   16.03]],
 'D_comp': [[27.31, 44.29, 22.0, 39.41, 55.08, 73.74],
  [57.58, 48.66, 42.45, 32.14, 63.56, 95.44],
  [72.53, 77.39, 73.6, 95.6, 65.31, 21.63],
 

In [95]:
I = set(range(0, config['i_amount']))  
J = set(range(0, config['j_amount']))  
K = set(range(0, config['k_amount']))  
L = set(range(0, config['l_amount']))  
'''
Decision Variables
===============================
y : Whether location j build facility or not, Dim: (j) , BINARY
x : The amount of resource k allocated to location j, Dim: (j,k), CONTINUOUS
aX : Extra attractiveness allocated to location j, Dim: (j), CONTINUOUS
===============================

Other Variables
u : Utility of facility j to customer point i, Dim:(i, j), CONTINUOUS
P : Probability of customer i choosing facility j, Dim : (i, j), CONTINUOUS
TA : Total Attractiveness for customer location i, Dim : (i), CONTINUOUS
===============================
'''

'\nDecision Variables\n===============================\ny : Whether location j build facility or not, Dim: (j) , BINARY\nx : The amount of resource k allocated to location j, Dim: (j,k), CONTINUOUS\naX : Extra attractiveness allocated to location j, Dim: (j), CONTINUOUS\n===============================\n\nOther Variables\nu : Utility of facility j to customer point i, Dim:(i, j), CONTINUOUS\nP : Probability of customer i choosing facility j, Dim : (i, j), CONTINUOUS\nTA : Total Attractiveness for customer location i, Dim : (i), CONTINUOUS\n===============================\n'

In [96]:
# Initialize the model
model = Model("Competitive Facility Location")
model.setParam('NumericFocus', 3)
y = model.addVars(J, vtype=GRB.BINARY, name="y") #3.14
x = model.addVars(J, K, vtype=GRB.CONTINUOUS, name="x", lb=0) # 3.12
aX = model.addVars(J, vtype=GRB.CONTINUOUS, name="aX", lb=0) # 3.13

u = model.addVars(I, J, vtype=GRB.CONTINUOUS, name="utility(u)", lb=0) # utility var(3.5)
P = model.addVars(I, J, vtype=GRB.CONTINUOUS, name="P, utility vs TA ratio", lb=0.0, ub=1.0) # Probability of i choosing j
TA = model.addVars(I, vtype=GRB.CONTINUOUS, name="TA", lb=0.00000000)
TAi_lambda = model.addVars(I, vtype=GRB.CONTINUOUS, name="negative TAi*lambda(e's power)", lb= -100000, ub= -0.000001) #-lambda*TAi
G_exp_vars = model.addVars(I, vtype=GRB.CONTINUOUS, name="G_exp_vars", lb=0.0, ub=1) #e^-(lambda*TAi), ub=0.5

model.update()

model.addConstrs((x[j, k] <= config['U_LT'][j][k] * y[j] for j in J for k in K), "(3.1) Resource only for built facility, and amount of type k for facility j is limit at U_LT[j, k]")
model.addConstrs((quicksum(x[j, k] for j in J) <= config['U_T'][k] for k in K), "(3.2) The total amount of resource k is U_T[k]")
model.addConstrs((quicksum(x[j, k] for k in K) <= config['U_L'][j] for j in J), "(3.3) The max resource sum for facility j is U_L[j]")
model.addConstrs((aX[j] <= config['M'] * y[j] for j in J), "(3.4) Extra Attractiveness Limit")

model.addConstrs((u[i, j] == ((quicksum(config['V'][j][k] * x[j, k] for k in K) + aX[j]) / (config['D'][i][j]**2)) for i in I for j in J), "(3.5) utility for facility j to customer i")
model.addConstrs((u[i, j] == P[i, j] * TA[i] for i in I for j in J), "(3.8) calculate P by uij/TAi, 分母要是常數所以用乘的")
## TA的限制式, 因為不能讓 G_exp中 addGenConstrExp內的變數為多個變數的線性組合所以要另外創TA變數
model.addConstrs((TA[i] == quicksum(u[i, j] for j in J) + (quicksum(config['A_opponent_bar'][l] / (config['D_comp'][i][l]**2) for l in L)) for i in I), "(3.7) calculate total TA by u and A_bar")

## G的計算
# def G(tai):
#     return 1 - np.exp(-config['lambda_for_G'] * tai)
model.addConstrs((TAi_lambda[i] == (-config['lambda_for_G'] * TA[i]) for i in I), "negative tai*lambda constr")
# Gurobi無法直接在目標式有exponential
for i in I:
    model.addGenConstrExpA(TAi_lambda[i], G_exp_vars[i], math.e, name=f"G_exp_constr_{i}")



# Objective function: Maximize profit by attracting customers - costs
objective = ((quicksum(config['H'][i] * (1 - G_exp_vars[i]) * (quicksum(P[i, j] for j in J)) for i in I)) \
          - (quicksum(config['F'][j] * y[j] + config['C'][j] * aX[j] + quicksum(config['B'][j][k] * x[j, k] for k in K) for j in J)))
model.setObjective(objective, GRB.MAXIMIZE)


# Solve and Output
model.optimize()
if model.status == GRB.OPTIMAL:
    print("Optimal solution found.")
else:
    print("No optimal solution found.")
    model.computeIIS()
    # 将IIS输出到文件
    # model.write("model_iis.mps")
    print("IIS written to file 'model_iis.mps'")



Set parameter NumericFocus to value 3
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 21.5.0 21F79)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 133 rows, 195 columns and 543 nonzeros
Model fingerprint: 0xc727e49f
Model has 60 quadratic objective terms
Model has 60 quadratic constraints
Model has 5 general constraints
Variable types: 183 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [3e-06, 1e+06]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 4e+01]
  QObjective range [4e+01, 8e+01]
  Bounds range     [1e-06, 1e+05]
  RHS range        [2e-01, 2e+01]
Presolve added 0 rows and 93 columns
Presolve removed 32 rows and 0 columns
Presolve time: 0.00s
Presolved: 462 rows, 350 columns, 1560 nonzeros
Presolved model has 5 SOS constraint(s)
Presolved model has 120 bilinear constraint(s)


         in product terms.
         Presolve was not able to compute smaller bounds for these variables.
         Consider bounding these variables or reformulating the model.


Solving non-convex MIQCP

Variable types: 338 continuous, 12 integer (12 binary)

Root relaxation: objective 1.326176e+03, 368 iterations, 0.02 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1326.17610    0  128          - 1326.17610      -     -    0s
H    0     0                      60.0374260 1321.17695  2101%     -    0s
     0     2 1321.17695    0  128   60.03743 1321.17695  2101%     -    0s
H  654   592                      60.0374260 1257.05051  1994%  43.6    0s
H 2020  1262                      60.0374261 1197.71714  1895%  38.1    4s
  3128  1732  396.98469  168  126   60.03743 1106.90410  1744%  42.1    5s
H11571  7322                      60.0374262  979.

GurobiError: Cannot compute IIS on a feasible model

In [92]:
for var in model.getVars():
    print(f"{var.varName} = {var.x}")
print("")

y[0] = -0.0
y[1] = -0.0
y[2] = -0.0
y[3] = 1.0
y[4] = 1.0
y[5] = -0.0
x[0,0] = 0.0
x[0,1] = 0.0
x[0,2] = 0.0
x[1,0] = 0.0
x[1,1] = 0.0
x[1,2] = 0.0
x[2,0] = 0.0
x[2,1] = 0.0
x[2,2] = 0.0
x[3,0] = 5.0
x[3,1] = 3.0
x[3,2] = 1.5863878120817373e-05
x[4,0] = 0.0
x[4,1] = 0.0
x[4,2] = 4.999985767433732
x[5,0] = 0.0
x[5,1] = 0.0
x[5,2] = 0.0
aX[0] = 0.0
aX[1] = 0.0
aX[2] = 0.0
aX[3] = 0.0
aX[4] = 0.0
aX[5] = 0.0
utility(u)[0,0] = 0.0
utility(u)[0,1] = 0.0
utility(u)[0,2] = 0.0
utility(u)[0,3] = 77511.21294225348
utility(u)[0,4] = 98341.57280942424
utility(u)[0,5] = 0.0
utility(u)[1,0] = 0.0
utility(u)[1,1] = 0.0
utility(u)[1,2] = 0.0
utility(u)[1,3] = 282065.27577648475
utility(u)[1,4] = 32022.410589227762
utility(u)[1,5] = 0.0
utility(u)[2,0] = 0.0
utility(u)[2,1] = 0.0
utility(u)[2,2] = 0.0
utility(u)[2,3] = 320045.65850767517
utility(u)[2,4] = 18230.90550619371
utility(u)[2,5] = 0.0
utility(u)[3,0] = 0.0
utility(u)[3,1] = 0.0
utility(u)[3,2] = 0.0
utility(u)[3,3] = 31553.505212194657
utili

# References

## G calculation substitude exponential

https://www.gurobi.com/documentation/10.0/refman/py_model_agc_exp.html

https://support.gurobi.com/hc/en-us/community/posts/360077178491-Build-an-objective-function-with-Log-and-Exponential-inside-

https://support.gurobi.com/hc/en-us/community/posts/10481250723217-How-to-add-log-and-exponential-term-in-objective-function
